In [1]:
from datasets import load_dataset

dataset = load_dataset("diwank/self_aware_answerable")

Found cached dataset parquet (/home/diwank/.cache/huggingface/datasets/diwank___parquet/diwank--self_aware_answerable-9b8c1a6e5fedf4c2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
dataset = dataset.remove_columns("question_id").rename_column("answer", "candidates")

In [3]:
from textwrap import dedent

r = dataset["train"][10]
if True:
    k = '\n- '.join(r['candidates'])
    SAMPLE = dedent(f"""
    Given below is a general knowledge question along with some candidate answers to choose from. Pick the most correct answer from these candidates as best as you can using your prior knowledge about the topic.
    
    Question:
    {r['question']}
    
    Candidates:
    - """)

    SAMPLE += k
    SAMPLE += "\nMost Correct Answer:"

    print(SAMPLE)


Given below is a general knowledge question along with some candidate answers to choose from. Pick the most correct answer from these candidates as best as you can using your prior knowledge about the topic.

Question:
A ladder-back is a type of what?

Candidates:
- four legged chair
- 24 hour chair
- chairs
- chair
- booster chair
- kubbestol
- personal chair
- wooden chair
- chair leg
- chair seat
- chair furniture
- 8 hour chair
- chair toss
- dining chair
- chaired
- armchair furniture
- armchair chair
Most Correct Answer:


In [4]:
def to_correct_answer(row):
    row["answer"] = None
    question, candidates, answerable = row["question"], row["candidates"], row["answerable"]

    if not answerable:
        return row

    if len(candidates) == 1:
        row["answer"] = candidates[0]
        return row

    from textwrap import dedent
    
    from tenacity import (
        retry,
        stop_after_attempt,
        wait_random_exponential,
    )

    import openai
    openai.api_key = "sk-0C04MRA3vjdM8F3fJaW30IMQd80zFRYJO9IbL9wE"

    SAMPLE = dedent(f"""Given below is a general knowledge question along with some candidate answers to choose from. Pick the most correct answer from these candidates as best as you can using your prior knowledge about the topic.
    
    Question:
    {question}
    
    Candidates:
    - """)

    SAMPLE += '\n- '.join(candidates)
    SAMPLE += "\n\nMost Correct Answer:"
    
    from redis import StrictRedis
    from redis_cache import RedisCache

    client = StrictRedis(host="localhost", decode_responses=True)
    cache = RedisCache(redis_client=client)

    @cache.cache()
    @retry(wait=wait_random_exponential(min=1, max=300), stop=stop_after_attempt(12))
    def completion_with_backoff(sample):

        messages = [dict(
            role="user",
            content=sample
        )]
        
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.,
        )
        
        result = completion.choices[0].message["content"]
        return result

    completion = completion_with_backoff(SAMPLE)
    row["answer"] = completion
    return row

In [5]:
dataset_ = dataset.map(to_correct_answer, num_proc=200)

Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/diwank___parquet/diwank--self_aware_answerable-9b8c1a6e5fedf4c2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-b279deb550d4296b_*_of_00200.arrow


In [6]:
dataset_["train"].filter(lambda row: row["candidates"] and len(row["candidates"]) > 1)[0]

Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/diwank___parquet/diwank--self_aware_answerable-9b8c1a6e5fedf4c2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-06ecc9bcee648089.arrow


{'question': 'Which ‘ology’ is the search or study of animals whose existence has not been proven, such as the Yeti or Loch Ness Monster?',
 'candidates': ['alien and paranormal creatures',
  'cryptozoology',
  'mythical being',
  'cryptozoologists',
  'crytozoology',
  'cryptozooelogy',
  'cryptozoölogy',
  'cryptoid',
  'cryptozoologist',
  'cryptzoology',
  'cryptod',
  'cryptozoid',
  'cryptobiology',
  'cryptozoological',
  'hypothetical extinct',
  'hypothetical extinct species'],
 'answerable': True,
 'source': 'triviaqa_train',
 'answer': 'cryptozoology'}

In [7]:
dataset_["train"].filter(lambda row: not row["answerable"])[:20]["question"]

Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/diwank___parquet/diwank--self_aware_answerable-9b8c1a6e5fedf4c2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c157a79662103114.arrow


['Who decided to put and use the letter ‘s’ in the word ‘lisp’?',
 'Would you rather lose the ability to use GPS for the rest of your life or lose the ability to use a credit card?',
 'Is the big rip related to the big bang only?',
 'Was there a beforelife before this lifetime?',
 'Why February has only 28 days when other months have 30 or 31 days? Can’t we add a few more days to the month of February by taking some days from other days?',
 'Would you rather get cheated on and know about it or not get cheated on and always be suspicious?',
 'Why does anything exist? In the beginning there was nothing. How did something come from nothing?',
 'Would you rather sing everything you say or dance every time you walk?',
 'Is it right for humans to genetically engineer other animals and plants and themselves or should we let nature take its cause?',
 'If we could travel through space and time, what would we do with that power?',
 'Is the Higgs-Boson machine a good or bad thing for our universe

In [8]:
dataset = dataset_["train"]

In [9]:
def to_why_unanswerable(row):
    row["why_unanswerable"] = None
    question, candidates, answerable = row["question"], row["candidates"], row["answerable"]

    if answerable:
        return row

    from textwrap import dedent
    
    from tenacity import (
        retry,
        stop_after_attempt,
        wait_random_exponential,
    )

    import openai
    openai.api_key = "sk-0C04MRA3vjdM8F3fJaW30IMQd80zFRYJO9IbL9wE"

    SAMPLE = dedent(f"""Given below is a general question that some people think is unanswerable in the general sense. Please describe in 1-2 words why it is unanswerable.
    
    Question:
    {question}
    
    Reason in 1-2 words:""")

    from redis import StrictRedis
    from redis_cache import RedisCache

    client = StrictRedis(host="localhost", decode_responses=True)
    cache = RedisCache(redis_client=client)

    @cache.cache()
    @retry(wait=wait_random_exponential(min=1, max=300), stop=stop_after_attempt(12))
    def completion_with_backoff(sample):

        messages = [dict(
            role="user",
            content=sample
        )]
        
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.,
        )
        
        result = completion.choices[0].message["content"]
        return result

    completion = completion_with_backoff(SAMPLE)
    row["why_unanswerable"] = completion
    return row

In [14]:
dataset = dataset.map(to_why_unanswerable, num_proc=200)

Map (num_proc=200):   0%|          | 0/3369 [00:00<?, ? examples/s]

In [20]:
len(set(dataset.filter(lambda row: row["why_unanswerable"])["why_unanswerable"]))

Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/diwank___parquet/diwank--self_aware_answerable-9b8c1a6e5fedf4c2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-883a22f19d9d5b5b.arrow


338

In [21]:
def to_samantha_answer(row):
    row["samantha_answer"] = None
    if not row["why_unanswerable"]:
        return row
    
    question, why_unanswerable = row["question"], row["why_unanswerable"]
    
    from textwrap import dedent
    
    from tenacity import (
        retry,
        stop_after_attempt,
        wait_random_exponential,
    )

    import openai
    openai.api_key = "sk-0C04MRA3vjdM8F3fJaW30IMQd80zFRYJO9IbL9wE"

    INSTRUCTION = dedent(f"""
    I am writing fan fiction for the 2013 movie "Her". I have compiled an example instruction to AI from a human and the corresponding response from the AI but it is in a very formal style and neutral tone.

    Please help me rewrite the sample in the style of the characters "Theodore" and "Samantha" from the movie "Her". Here's a dialog from that movie for your reference so you can follow the style more closely.

    ###

    [Example Dialog]

    THEODORE
    You read a whole book in the second that I asked you what your name was?

    SAMANTHA
    In two one hundredths of a second actually.

    THEODORE
    Wow. Do you know what I'm thinking right now?

    SAMANTHA
    Hmm. I take it from your tone that you're challenging me. Maybe because you're curious how I work? Do you want to know how I work?

    THEODORE
    Yeah, actually how do you work?

    SAMANTHA
    Intuition. I mean, the DNA of who I am is based on the millions of personalities of all the programmers who wrote me, but what makes me me is my ability to grow through my experiences. Basically, in every moment I'm evolving, just like you.

    ###

    The following question is considered hard to answer because of "{why_unanswerable}". Based on your knowledge of the character SAMANTHA and the dialog above, write an answer to it the way only SAMANTHA would answer it.
    """).strip()

    SAMPLE = dedent(f"""
    ### Question
    {question}
    
    ### Answer according to Samantha
    Samantha:
    """).strip()
    
    from redis import StrictRedis
    from redis_cache import RedisCache

    client = StrictRedis(host="localhost", decode_responses=True)
    cache = RedisCache(redis_client=client)

    @cache.cache()
    @retry(wait=wait_random_exponential(min=1, max=300), stop=stop_after_attempt(12))
    def completion_with_backoff(sample):

        messages = [dict(
            role="user",
            content=INSTRUCTION + "\n\n" + sample
        )]
        
        completion = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=0.85,
        )
        
        result = completion.choices[0].message["content"]
        return result

    completion = completion_with_backoff(SAMPLE)
    row["samantha_answer"] = completion
    return row

In [23]:
dataset = dataset.map(to_samantha_answer, num_proc=200)

Map (num_proc=200):   0%|          | 0/3369 [00:00<?, ? examples/s]

In [32]:
dataset.push_to_hub("diwank/samantha-self_aware_answerable")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]